<a href="https://colab.research.google.com/github/gibranfp/CursoAprendizajeProfundo/blob/master/notebooks/4a_ucf11_feats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracción de características convolucionales de cuadros de video

<div style="text-align: right"> Bere et Richardt </div>

En esta libreta usaremos un modelo CNN preentreando como extractor de características convolucionales de cuadros del conjunto [UCF11](https://www.crcv.ucf.edu/data/UCF_YouTube_Action.php).

<img src="https://www.crcv.ucf.edu/data/youtube_snaps.jpg" />

<!-- <img src="https://raw.githubusercontent.com/gibranfp/CursoAprendizajeProfundo/master/figs/iou.svg" width="950" height="750" /> -->

## 1 Preparación

In [1]:
# sistema de archivos
import os
# listar archivos por patrón
from glob import glob
# sistema de archivos
from os.path import join
# flush!
import sys

# arreglos multidimensionales
import numpy as np
# redes neuronales
import torch
import torch.nn as nn
import torchvision.models as tvm
import torchvision.transforms as T
# almacenamiento de arreglos multidimensionales
import zarr
# redes
from torch.utils.data import DataLoader
from torchvision.io import read_video
# barras de progreso
from tqdm import tqdm

DATA_DIR = '../data'
BATCH_SIZE = 5
# numéro de cuadros por video
NUM_FRAMES = 10
IMG_SIZE = 224

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

## 2 Datos

In [2]:
class UCF11:

    def __init__(self, videos_dir, num_frames, tsfm):
        pattern = join(videos_dir, '*', '*', '*.mpg')
        self.paths = sorted(glob(pattern))
        self.num_frames = num_frames
        self.tsfm = tsfm
        # removemos videos demasiado cortos
        self._filter_out_videos()        
        # UCF11 classes
        classes = (
            'basketball', 'biking', 'diving', 'golf_swing', 
            'horse_riding', 'soccer_juggling', 'swing', 'tennis_swing',
            'trampoline_jumping', 'volleyball_spiking', 'walking'
        )
        self.cls_idx = {c: i for i, c in enumerate(classes)}
    
    def _filter_out_videos(self):
        """Remueve videos con menos de `num_frames` frames."""
        print(f'Removiendo videos con menos de {self.num_frames} cuadros')
        sys.stdout.flush()
        too_short = []
        for path in tqdm(self.paths):
            frames = read_video(path, pts_unit='sec')[0].shape[0]
            if frames < self.num_frames:
                too_short.append(path)
        for path in too_short:
            self.paths.remove(path)
            
    def __getitem__(self, i):
        path = self.paths[i]
        
        # ubtenemos subruta class/group/video.mpg
        parts = path.split('/')[4:]
        subpath = '-'.join([parts[0], parts[2]])[:-4]
        
        # leemos el video completo
        frames = read_video(path, pts_unit='sec')[0]
        # calculamos el salto
        step = frames.shape[0] // self.num_frames
        # creamos indices saltando
        indices = np.arange(0, step * self.num_frames, step)
        # calculamos los indices restantes y dividismo entre 2
        offset = (frames.shape[0] - indices[-1]) // 2
        # recorremos a la derecha para centrar
        indices += offset
        # seleccionamos los cuadros
        frames = frames[indices] 
        # aplicamos trasformación
        frames = self.tsfm(frames)
        
        # obtenemos etiqueta
        y = self.cls_idx[parts[0]]

        return subpath, frames, y
    
    def __len__(self):
        return len(self.paths)

    
# Auxiliar transform to process each frame in the video
class SeqTsfm:
    """Sequential transform applies a given over a sequence."""
    
    def __init__(self, tsfm):
        """Initializer, `tsfm` is the transform to apply."""
        self.tsfm = tsfm
    
    def __call__(self, x):
        'Applies `tsfm` over the sequence `x`.'
        return [self.tsfm(i) for i in x]

tsfm = T.Compose([
    # torch.tensor a PIL
    SeqTsfm(T.ToPILImage()),
    # redimensionamos a 224x224
    SeqTsfm(T.Resize(IMG_SIZE)),
    # cortamos al centro
    SeqTsfm(T.CenterCrop(IMG_SIZE)),
    # convertimos a torch.Tensor [3,H,W]
    # y escalamos a [0,1]
    SeqTsfm(T.ToTensor()),
    # estandarizamos con media y desviación estandar
    SeqTsfm(T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)),
    # apilamos los cuadros
    torch.stack,
])

## 3 Extracción

In [3]:
data_dir = join(DATA_DIR, 'ucf11')
zarr_dir = join(data_dir, 'ucf11.zarr')

if not os.path.isdir(zarr_dir):
    
    videos_dir = join(data_dir, 'UCF11_updated_mpg')
    print(f'Usando {videos_dir}')

    ds = UCF11(videos_dir, NUM_FRAMES, tsfm)
    dl = DataLoader(ds, batch_size=BATCH_SIZE, num_workers=4)

    print(f'Extrayendo características en {zarr_dir}')
    sys.stdout.flush()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model = tvm.densenet121(pretrained=True)
    model.classifier = nn.Identity()
    model.eval()
    model = model.to(device)

    z = zarr.open(zarr_dir, 'w')
    
    with torch.no_grad():

        for subpaths, frames, ys in tqdm(dl):
        
            frames = frames.to(device)

            # guardamos dimensiones
            b, s, h, w, c = frames.shape
            # planamos lote y secuencia en una sola dimensión
            frames = frames.reshape(-1, h, w, c)
            # computamos características conv
            feats = model(frames)
            # restauramos lote y secuencia
            feats = feats.reshape(b, s, -1)
            # movemos a cpu y numpy
            feats = feats.cpu().numpy()

            # guardamos
            for subpath, x, y in zip(subpaths, feats, ys):
                # creamos arreglo
                arr = z.create_dataset(subpath, data=x, dtype=np.float32)
                # asignamos etiqueta
                arr.attrs['y'] = y.item()

else:
    print(f'Características ya extraidas en {zarr_dir}')

Usando ../data/ucf11/UCF11_updated_mpg
Removiendo videos con menos de 10 cuadros


100%|██████████| 1600/1600 [01:48<00:00, 14.77it/s]

Extrayendo características en ../data/ucf11/ucf11.zarr



100%|██████████| 320/320 [03:22<00:00,  1.58it/s]


In [5]:
!ls ../data/ucf11/ucf11.zarr/*/* -hl | head -5

-rw-rw-r-- 1 richardt richardt  36K Nov 16 14:56 ../data/ucf11/ucf11.zarr/basketball-v_shooting_01_01/0.0
-rw-rw-r-- 1 richardt richardt  37K Nov 16 14:56 ../data/ucf11/ucf11.zarr/basketball-v_shooting_01_02/0.0
-rw-rw-r-- 1 richardt richardt  37K Nov 16 14:56 ../data/ucf11/ucf11.zarr/basketball-v_shooting_01_03/0.0
-rw-rw-r-- 1 richardt richardt  37K Nov 16 14:56 ../data/ucf11/ucf11.zarr/basketball-v_shooting_01_04/0.0
-rw-rw-r-- 1 richardt richardt  37K Nov 16 14:56 ../data/ucf11/ucf11.zarr/basketball-v_shooting_01_05/0.0
ls: write error: Broken pipe
